# Data Exploration of Recipes Dataset

In [1]:
# Imports

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [17]:
# Loading the data
file = "../raw_data/recipes.csv"
recipes_df = pd.read_csv(file)

custom_stopwords_df = pd.read_csv("custom_stopwords.csv")
custom_stopwords = list(custom_stopwords_df['custom_stopwords'])

## Data sanity checks

In [3]:
recipes_df.describe()

,Unnamed: 0
count,13501.000000
mean,6750.000000
std,3897.547327
min,0.000000
25%,3375.000000
50%,6750.000000
75%,10125.000000
max,13500.000000


In [4]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           13501 non-null  int64 
 1   Title                13496 non-null  object
 2   Ingredients          13501 non-null  object
 3   Instructions         13493 non-null  object
 4   Image_Name           13501 non-null  object
 5   Cleaned_Ingredients  13501 non-null  object
dtypes: int64(1), object(5)
memory usage: 633.0+ KB


In [5]:
recipes_df.head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


## Checking for nulls and na

In [6]:
recipes_df.isna().sum()

Unnamed: 0             0
Title                  5
Ingredients            0
Instructions           8
Image_Name             0
Cleaned_Ingredients    0
dtype: int64

In [7]:
recipes_df[recipes_df["Title"].isna()]

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
11221,11221,NaN,[],NaN,roasted-game-hens-with-caramelized-root-vegeta...,['']
12373,12373,NaN,[],NaN,chicken-soup-with-rice-232605,['']
12378,12378,NaN,[],NaN,double-lemon-bars-232572,['']
12818,12818,NaN,[],NaN,pear-and-frangipane-crostata-with-raspberry-vi...,['']
12829,12829,NaN,[],NaN,hazelnut-shortbread-sticks-231311,['']


In [8]:
recipes_df[recipes_df["Instructions"].isna()]

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
4293,4293,Broccolini-Cheddar Gratin with Rye Breadcrumbs,"['1/4 loaf seeded rye bread, torn into 1"" piec...",NaN,broccolini-cheddar-gratin-with-rye-breadcrumbs...,"['1/4 loaf seeded rye bread, torn into 1"" piec..."
9636,9636,Smoked Salmon with Egg Salad and Green beans,[],NaN,smoked-salmon-with-egg-salad-and-green-beans-3...,['']
10356,10356,Royal Icing,"['Using electric mixer, beat 3 1/4 cups powder...",NaN,royal-icing-240751,"['Using electric mixer', 'beat 3 1/4 cups powd..."
11221,11221,NaN,[],NaN,roasted-game-hens-with-caramelized-root-vegeta...,['']
12373,12373,NaN,[],NaN,chicken-soup-with-rice-232605,['']
12378,12378,NaN,[],NaN,double-lemon-bars-232572,['']
12818,12818,NaN,[],NaN,pear-and-frangipane-crostata-with-raspberry-vi...,['']
12829,12829,NaN,[],NaN,hazelnut-shortbread-sticks-231311,['']


## Cleaning the data

In [9]:
recipes_df = recipes_df.drop(columns ='Unnamed: 0')
recipes_df.dropna

<bound method DataFrame.dropna of                                                    Title  \
0      Miso-Butter Roast Chicken With Acorn Squash Pa...   
1                        Crispy Salt and Pepper Potatoes   
2                            Thanksgiving Mac and Cheese   
3                     Italian Sausage and Bread Stuffing   
4                                           Newton's Law   
...                                                  ...   
13496                               Brownie Pudding Cake   
13497  Israeli Couscous with Roasted Butternut Squash...   
13498  Rice with Soy-Glazed Bonito Flakes and Sesame ...   
13499                                        Spanakopita   
13500  Mexican Poblano, Spinach, and Black Bean "Lasa...   

                                             Ingredients  \
0      ['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...   
1      ['2 large egg whites', '1 pound new potatoes (...   
2      ['1 cup evaporated milk', '1 cup whole milk', ...   
3    

In [10]:
# Calculating the length of 'Cleaned_Ingredients' columns to identify any empty ingredients and then dropping them

recipes_df['clean_len'] = [len(i) for i in recipes_df["Cleaned_Ingredients"]]
recipes_df.drop(recipes_df[recipes_df['clean_len']<5].index, axis = 0, inplace= True)

In [11]:
# Function to get rid of punctuation, numbers and weird formatting

def clean_list(ingredient_list):
    '''function to clean ingredints list'''
    punctuation = string.punctuation
    
    # break string into list of individual items
    ingredient_list = ingredient_list.split("', \'")

    # iterate through each item in list to remove punctuation and non alpha characters
    for i in range(len(ingredient_list)):
        for punc in punctuation:
            ingredient_list[i] = ingredient_list[i].replace(punc, '')
        ingredient_list[i] = ''.join(char for char in ingredient_list[i] if char.isalpha() or char == ' ')
        ingredient_list[i] = ingredient_list[i].strip()
        ingredient_list[i] = ingredient_list[i].replace('  ', ' ')
        
    return ' '. join(ingredient_list)

recipes_df['Clean2'] = recipes_df['Cleaned_Ingredients'].map(clean_list)

## Bag of Word and Stop Words

In [12]:
len(custom_stopwords)

293

In [13]:
# Setting bag of words scope - first 500

bag_of_ingredients = recipes_df["Clean2"][:2000]

In [14]:
# Removing regular stopwords

stop_words = set(stopwords.words('english')) 

for i in range(0, len(bag_of_ingredients)):
    word_tokens = word_tokenize(bag_of_ingredients[i])
    word_tokens = [w.lower() for w in word_tokens]
    full_stop_words = stop_words | set(custom_stopwords)
    bag_of_ingredients[i] = [w for w in word_tokens if not w in full_stop_words]

# Converting list to string
bag_of_ingredients = bag_of_ingredients.map(' '.join)

In [15]:
#Creating the bag of words dataframe

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(bag_of_ingredients)

bag_of_ingredients_df = pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names_out())

In [16]:
#Doing a count of the bag of words to identify custom stop words
#Denise is doing n00: n50 (Denise up to 850)

bag_of_ingredients_df.sum().sort_values(ascending=False)[0:50] # Starting with the first 50 rows 


oil            1515
ground         1271
pepper         1156
fresh           965
olive           902
garlic          822
black           715
red             701
butter          660
juice           653
leaves          650
lemon           647
extravirgin     616
cloves          564
white           517
vinegar         495
seeds           494
flour           405
powder          394
onion           393
vegetable       387
sauce           367
cream           337
lime            331
rice            315
dried           310
chicken         309
wine            295
allpurpose      293
eggs            283
green           281
vanilla         269
milk            259
coconut         258
ginger          255
brown           242
cilantro        237
baking          235
egg             231
cheese          228
parsley         225
tomatoes        225
dry             220
extract         212
honey           210
sesame          202
chile           170
clove           169
granulated      169
cinnamon        166
